In [1]:
import numpy as np
import pandas as pd
import pygsheets
from ds.utilities.io import ds_trino as ds_presto
import json
import pandas_gbq
from google.oauth2 import service_account
import requests

In [2]:
sql = """

DROP TABLE IF EXISTS hudi_ingest.scratch.cr_starts

"""

try:
    df = ds_presto.fetch_data(sql_str=sql, use_cache = False)
except TypeError:
    pass

sql = """



CREATE TABLE hudi_ingest.scratch.cr_starts AS

WITH

  ss_orgs_raw AS (
    SELECT
      CAST(transaction.org_uuid AS VARCHAR) AS brand_organization_id
      ,CAST(inventory.facility_uuid AS VARCHAR) AS facility_id
      ,MIN(DATE(transaction.created_at)) FILTER (WHERE transaction.type = 'TRANSACTION_RECEIVED') AS org_fac_first_rec
      ,MIN(DATE(transaction.created_at)) FILTER (WHERE transaction.type = 'TRANSACTION_SOLD') AS org_fac_first_sale
    FROM
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction_line_item AS line_item
    JOIN
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory AS inventory
      ON line_item.inventory_uuid = inventory.uuid
    JOIN
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction AS transaction
      ON line_item.inventory_transaction_uuid = transaction.uuid
    WHERE 
      line_item.day_partition IS NOT NULL
      AND inventory.day_partition IS NOT NULL
      AND transaction.day_partition IS NOT NULL
      AND line_item.applied_stock_on_hand_delta IS NOT NULL
      AND (transaction.type = 'TRANSACTION_RECEIVED' OR transaction.type = 'TRANSACTION_SOLD')
    GROUP BY
      1,2
  )
  
  ,ss_stores AS (
    SELECT
      brand_organization_id AS prep_organization_id
      ,facility_id
      ,IF(org_fac_first_sale IS NOT NULL AND org_fac_first_rec IS NOT NULL, GREATEST(org_fac_first_sale, org_fac_first_rec), org_fac_first_sale) AS org_fac_first_sale
      ,COALESCE(org_fac_first_rec, org_fac_first_sale) AS org_fac_first_rec
      , store_id
    FROM
      hudi_ingest.data_science.ds_stores_view
    JOIN
      ss_orgs_raw
      USING(brand_organization_id, facility_id)
  )
  
 ,ss_starts AS (
    SELECT
      prep_organization_id
      ,facility_id
      ,DATE(MIN(created_at)) AS org_fac_start_date
    FROM
      hudi_ingest.store_availability.store_state_changes_view
    JOIN
      ss_stores
       ON  hudi_ingest.store_availability.store_state_changes_view.store_id = CAST(ss_stores.store_id AS VARCHAR)
       
    WHERE --Makes sure the store opened organically + automatically – without human intervention – indicating that we've finished testing, and now it's up and running
      NOT store_open_before
      AND store_open_now
      AND store_should_be_open_now
      AND store_closure_reason_now IS NULL
      AND store_closure_reason_before = 'OFF_HOUR'
      AND created_at > org_fac_first_rec
      AND day_partition <= cast(current_date AS VARCHAR)
    GROUP BY
      1,2
  )

SELECT
  *
FROM (
  SELECT
    prep_organization_id
    ,facility_id
    ,IF(org_fac_first_sale IS NOT NULL AND org_fac_start_date IS NOT NULL, LEAST(org_fac_first_sale, org_fac_start_date), COALESCE(org_fac_start_date, org_fac_first_sale)) AS org_fac_live_date
  FROM
    ss_stores
  LEFT JOIN
    ss_starts
    USING(prep_organization_id, facility_id)
  GROUP BY
    1,2,3
)
WHERE
  org_fac_live_date IS NOT NULL

  """

df = ds_presto.fetch_data(sql_str=sql, use_cache = False)
df.head()



,rows
0,5161
